In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def normalize(arr,cou):
  sum_elements=np.sum(arr)
  if(sum_elements!=0):
   mean=sum_elements/cou
   d=arr-mean
   std=np.sum(np.multiply(d,d))
   stdev=(std/cou)**0.5
   arr=(arr-mean)*(1/stdev)
  return arr

In [5]:
def get_data(temp):
 y_train=temp['1']
 x_train=temp
 del x_train['1']

 X_train=x_train.to_numpy()
 Y_train=y_train.to_numpy()
 r,c=X_train.shape
 #X_train=np.concatenate((np.ones((r,1)),X_train),axis=1)
 
 X_train=X_train.T
 for i in range(c):
  if(i==0):
   continue 
  X_train[i]=normalize(X_train[i],r)
 X_train=X_train.T
 
 return X_train,Y_train

In [6]:
temp= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emnist-letters-train (1).csv')
temp1=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emnist-letters-test.csv')

y1 = temp1['1']
Y1 = np.array(y1)
j = 5
x,y=get_data(temp)
X = np.array(pd.DataFrame(x))
Y = np.array(pd.DataFrame(y))
x_train = X
y_train = Y

x1,y1=get_data(temp1)
X1 = np.array(pd.DataFrame(x1))
Y1 = np.array(pd.DataFrame(y1))
x_val = X1
y_val = Y1

(88800, 784)


In [18]:
class NN():
    def __init__(self, sizes, epochs=3, l_rate=0.001):
        self.sizes = sizes
        self.epochs = epochs
        self.l_rate = l_rate
        self.params = self.initialization()

    def sigmoid(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

    def softmax(self, x, derivative=False):
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)

    def initialization(self):
        input_layer=self.sizes[0]
        hidden_1=self.sizes[1]
        hidden_2=self.sizes[2]
        output_layer=self.sizes[3]
        np.random.seed(42)

        params = {
            'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
            'W2':np.random.randn(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
            'W3':np.random.randn(output_layer, hidden_2) * np.sqrt(1. / output_layer)
        }

        return params

    def forward_pass(self, x_train):
        params = self.params
        params['A0'] = x_train
        params['Z1'] = np.dot(params["W1"], params['A0'])
        params['A1'] = self.sigmoid(params['Z1'])
        params['Z2'] = np.dot(params["W2"], params['A1'])
        params['A2'] = self.sigmoid(params['Z2'])
        params['Z3'] = np.dot(params["W3"], params['A2'])
        params['A3'] = self.softmax(params['Z3'])

        return params['A3']

    def backward_pass(self, y_train, output):
        params = self.params
        change_w = {}
        error = 2 * (output - y_train) / output.shape[0] * self.softmax(params['Z3'], derivative=True)
        change_w['W3'] = np.outer(error, params['A2'])
        error = np.dot(params['W3'].T, error) * self.sigmoid(params['Z2'], derivative=True)
        change_w['W2'] = np.outer(error, params['A1'])
        error = np.dot(params['W2'].T, error) * self.sigmoid(params['Z1'], derivative=True)
        change_w['W1'] = np.outer(error, params['A0'])

        return change_w

    def update_network_parameters(self, changes_to_w):
        for key, value in changes_to_w.items():
            self.params[key] -= self.l_rate * value

    def compute_accuracy(self, x_val, y_val):
        predictions = []

        for x, y in zip(x_val, y_val):
            output = self.forward_pass(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))
        
        return np.mean(predictions)*2

    def train(self, x_train, y_train, x_val, y_val):
        for iteration in range(self.epochs):
            for x,y in zip(x_train, y_train):
                output = self.forward_pass(x)
                changes_to_w = self.backward_pass(y, output)
                self.update_network_parameters(changes_to_w)
            
            accuracy = self.compute_accuracy(x_val, y_val)
            print('Epoch: {0}, Accuracy: ', accuracy * 100)

In [19]:
dnn = NN(sizes=[784, 128, 64, 10])
dnn.train(x_train, y_train, x_val, y_val)

KeyboardInterrupt: ignored